In [34]:
import math
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as mpimg # mpimg 用于读取图片
from torch.autograd import Variable
from PIL import Image

In [35]:
def ImageToTensor(path):
    RGBimg = np.array(Image.open(path).convert("RGB"))
    # print(RGBimg.shape)
    RGBimg = np.transpose(RGBimg, (2, 0, 1)) # 将通道放到第一维
    # print(RGBimg.shape)
    torchImg = torch.from_numpy(RGBimg)
    # print(torchImg.shape)
    torchImg = Variable(torch.unsqueeze(torchImg, dim=0).float(), requires_grad=False)
    # print(torchImg.shape)
    return torchImg

In [36]:
model = torchvision.models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1]) # 去除最后的fc分类层

pretrained_dict = torch.load('C:/Users/Lenovo/Desktop/ResNet_no_adaptation.pth', map_location='cpu')
model_dict = model.state_dict()

for k in model_dict.keys():
    if(('module.'+k) in pretrained_dict.keys()):
        model_dict[k]=pretrained_dict.get(('module.'+k))
model.load_state_dict(model_dict)
model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [37]:
images = ImageToTensor('./pkq.png')
images1 = ImageToTensor('./2.jpg')
images2 = ImageToTensor('./pkq2.png')

outputs1 = model(images1)
outputs1 = outputs1.detach().numpy()
outputs1 = outputs1.flatten()
# outputs1
# print(len(outputs))
# print(outputs)

outputs = model(images)
outputs = outputs.detach().numpy()
outputs = outputs.flatten()
# outputs

outputs2 = model(images2)
outputs2 = outputs2.detach().numpy()
outputs2 = outputs2.flatten()
# outputs2

In [38]:
def MSE_loss(feature1, feature2):
    loss = 0
    if(len(feature1) == len(feature2)):
        for i in range(0, len(feature1)):
            loss += math.pow((feature1[i] - feature2[i]), 2)
        loss /= len(feature1)
    return loss

In [39]:
loss_0_1 = MSE_loss(outputs, outputs1)
loss_0_2 = MSE_loss(outputs, outputs2)
print(loss_0_1, loss_0_2, loss_0_1 - loss_0_2)


0.0016182039541321248 0.0006847854901492 0.0009334184639829248
